In [217]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [248]:
# Set parameters
num_classes = 13 # Number of activities
num_features =102 # Number of features per sample
num_filters = 32 # Number of filters for convolutional layers
kernel_size = 2 # Size of kernel for convolutional layers
pool_size = 2 # Size of pool for max pooling layers
dropout_rate = 0.2 # Dropout rate for regularization
learning_rate = 0.001 # Learning rate for optimizer
num_epochs = 20 # Number of epochs for training
batch_size = 400 # Batch size for training

In [249]:
# Load and preprocess data
data = pd.DataFrame() # Create an empty dataframe
for subject in range(40, 58): # Loop over subjects (S40 to S57)
    if subject == 43 or subject == 45:
        continue
    for activity in range(1, 14): # Loop over activities (A01 to A13)
        file_name = f"dataset/videoandimus/S{subject}/S{subject}_A{activity:02d}.csv" # Create file name
        df = pd.read_csv(file_name) # Read file as dataframe
        df["subject"] = subject # Add subject column
        df["activity"] = activity # Add activity column
        data = pd.concat([data, df]) # Append dataframe to data

In [250]:
# Encode activity labels as integers
le = LabelEncoder()
data["activity"] = le.fit_transform(data["activity"])

# Split data into features (X) and labels (y)
X = data.drop(["subject", "activity"], axis=1).values # Drop subject and activity columns and convert to numpy array
y = data["activity"].values # Convert activity column to numpy array


In [251]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Use 10% of data for testing

# Reshape data to fit the input shape of the CNN model (samples, features, channels)
X_train = X_train.reshape(-1, num_features, 1) # Add channel dimension of 1
X_test = X_test.reshape(-1, num_features, 1) # Add channel dimension of 1

# Convert labels to one-hot vectors using Keras utility function
y_train = keras.utils.to_categorical(y_train, num_classes) # Convert to one-hot vectors with num_classes columns
y_test = keras.utils.to_categorical(y_test, num_classes) # Convert to one-hot vectors with num_classes columns


In [252]:
# Define CNN model using Keras sequential API
model = keras.Sequential()
model.add(layers.Conv1D(num_filters, kernel_size, activation="relu", input_shape=(num_features, 1))) # Convolutional layer with relu activation
model.add(layers.MaxPooling1D(pool_size)) # Max pooling layer
model.add(layers.Conv1D(num_filters*2, kernel_size, activation="relu")) # Convolutional layer with relu activation
model.add(layers.MaxPooling1D(pool_size)) # Max pooling layer
model.add(layers.Flatten()) # Flatten layer to convert 3D output to 1D
model.add(layers.Dropout(dropout_rate)) # Dropout layer for regularization
model.add(layers.Dense(num_classes, activation="softmax")) # Dense layer with softmax activation for output

In [253]:
# Compile model with loss function, optimizer and metrics
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate), metrics=["accuracy"])

# Fit model on training data
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=1)

# Evaluate model on testing data
y_pred = model.predict(X_test) # Predict labels for testing data
y_pred = y_pred.argmax(axis=1) # Convert one-hot vectors to integers
y_test = y_test.argmax(axis=1) # Convert one-hot vectors to integers

Epoch 1/20
360/360 [==============================] - 17s 44ms/step - loss: 65.5286 - accuracy: 0.2215
Epoch 2/20
360/360 [==============================] - 16s 43ms/step - loss: 1.8242 - accuracy: 0.3046
Epoch 3/20
360/360 [==============================] - 15s 43ms/step - loss: 1.5836 - accuracy: 0.3546
Epoch 4/20
360/360 [==============================] - 16s 45ms/step - loss: 1.4563 - accuracy: 0.3854
Epoch 5/20
360/360 [==============================] - 15s 42ms/step - loss: 1.3632 - accuracy: 0.4173
Epoch 6/20
360/360 [==============================] - 15s 42ms/step - loss: 1.1910 - accuracy: 0.4821
Epoch 7/20
360/360 [==============================] - 15s 42ms/step - loss: 1.0880 - accuracy: 0.5152
Epoch 8/20
360/360 [==============================] - 15s 42ms/step - loss: 1.0314 - accuracy: 0.5383
Epoch 9/20
360/360 [==============================] - 15s 42ms/step - loss: 1.0058 - accuracy: 0.5495
Epoch 10/20
360/360 [==============================] - 15s 43ms/step - loss: 0.97

In [254]:
# Print performance metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average="macro"))
print("Recall:", recall_score(y_test, y_pred, average="macro"))
print("F1-score:", f1_score(y_test, y_pred, average="macro"))

Accuracy: 0.6613662313069147
Precision: 0.7303945927495222
Recall: 0.69753207080066
F1-score: 0.6979203572863144
